In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import numpy as np
import torch
from torch import Tensor
import sigkernel
import os
import sys
import tslearn
import tslearn.metrics
import ksig
from tqdm import tqdm
import aeon

from features.random_fourier import RBF_RandomFourierFeatures
from features.random_sig_fourier import SigRBFTensorizedRandProj, SigVanillaTensorizedRandProj
from features.signature import SigTransform, LogSigTransform
from features.randomized_sig import RandomizedSignature
from features.random_warping_series import RandomWarpingSeries


#####################################################################
################## Print torch tensors ##############################
#####################################################################
import inspect

def mod_retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]


def print_shape(X):
    """Prints the name and shape of an array."""
    print(X.shape, mod_retrieve_name(X)[0], "\n")


def print_tensor(X):
    """Prints the name and shape of an array, then the array itself."""
    print(X.shape, mod_retrieve_name(X)[0])
    print(X, "\n")

# test aoen toolkit

In [2]:
# Print the different datasets
from aeon.datasets.tsc_datasets import multivariate, univariate
from aeon.datasets import load_classification

def get_aeon_dataset(
        dataset_name:str, 
        ):
    """Loads a dataset from the UCR/UEA archive using 
    the aeon library.

    Args:
        dataset_name (str): Name of the dataset

    Returns:
        Tuple: 4-tuple of the form (X_train, y_train, X_test, y_test)
    """
    X_train, y_train = load_classification(dataset_name, split="train")
    X_test, y_test = load_classification(dataset_name, split="test")

    return X_train.transpose(0, 2, 1), y_train, X_test.transpose(0, 2, 1), y_test


datasets_to_test = ["EthanolConcentration", "Heartbeat"]
# for dataset_name in datasets_to_test:
#     X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
#     print(f"Dataset: {dataset_name}")
#     print(f"X_train shape: {X_train.shape}")
#     print(f"y_train shape: {y_train.shape}")
#     print(f"X_test shape: {X_test.shape}")
#     print(f"y_test shape: {y_test.shape}")
#     print("\n")


# obtain minirocket results for univariate time series

In [3]:
from aeon.benchmarking import get_estimator_results_as_array
from aeon.datasets.tsc_datasets import univariate, univariate_equal_length
from aeon.registry import all_estimators

# est = all_estimators("classifier", filter_tags={"algorithm_type": "convolution"})
# for c in est:
#     print(c)
# names = [t[0].replace("Classifier", "") for t in est]
# # names.append("MiniROCKET")  # Alternatve configuration of the RocketClassifier


# Start here

In [4]:
from aeon.transformations.collection.convolution_based import MiniRocket
from aeon.datasets import load_unit_test
from preprocessing.stream_transforms import z_score_normalize
X_train, y_train = load_unit_test(split="train")
X_test, y_test = load_unit_test(split="test")
X_train, X_test = z_score_normalize(X_train, X_test)
print_shape(X_test)

trf = MiniRocket(num_kernels=512)
trf.fit(X_train)
feat_X_train = trf.transform(X_train)
feat_X_test = trf.transform(X_test)

feat_X_train.shape

(22, 1, 24) X_test 



(20, 504)

In [5]:
from aeon.transformations.collection.convolution_based import MiniRocketMultivariate
from preprocessing.stream_transforms import z_score_normalize, normalize_streams, augment_time, add_basepoint_zero
from aeon.classification.sklearn import RotationForestClassifier
from sklearn.linear_model import RidgeCV
from sklearn.metrics import accuracy_score
import time

def train_test_EC(transformer, 
                  transpose_time:bool = False, 
                  torch_input:bool = True,
                  device="cuda",
    ):
    with torch.no_grad():
        # load data
        torch.cuda.empty_cache()
        train_X, train_y, test_X, test_y = get_aeon_dataset("NATOPS")
        # train_X = np.diff(train_X, axis=1)
        # test_X = np.diff(test_X, axis=1)
        train_X, test_X = normalize_streams(train_X, test_X, max_T=100)
        if transpose_time:
            train_X, test_X = train_X.transpose(0, 2, 1), test_X.transpose(0, 2, 1)
        if torch_input:
            train_X = torch.tensor(train_X).to(device)
            train_X = add_basepoint_zero(train_X)
            train_X = augment_time(train_X)
            test_X  = torch.tensor(test_X).to(device)
            test_X = add_basepoint_zero(test_X)
            test_X  = augment_time(test_X)



        # fit transformer
        t0 = time.time()
        transformer.fit(train_X)
        train_X = transformer.transform(train_X)
        test_X = transformer.transform(test_X)
        if torch_input:
            train_X = train_X.cpu().numpy()
            test_X = test_X.cpu().numpy()
        t1 = time.time()
        print_shape(train_X)
        print(f"Time to transform: {t1-t0} seconds")

        # train classifier      
        clf = RotationForestClassifier()
        clf.fit(train_X, train_y)
        t2 = time.time()
        print(f"Time to fit classifier on train: {t2-t1} seconds")

        # predict
        pred = clf.predict(test_X)
        acc = accuracy_score(test_y, pred)
        t3 = time.time()
        print(f"Time to predict: {t3-t2} seconds")
        print(f"{acc} accuracy for {transformer}")

In [ ]:
train_test_EC(
    MiniRocketMultivariate(num_kernels=500),
    transpose_time=True,
    torch_input=False,
)
# Time to transform: 1.1914794445037842 seconds
# Time to fit classifier on train: 43.77142024040222 seconds
# Time to predict: 2.8060500621795654 seconds
# 0.39543726235741444 accuracy for MiniRocketMultivariate(num_kernels=500)

In [6]:
train_test_EC(
    RandomizedSignature(
        n_features=500,
        activation="linear"
    ),
    device="cuda",
)

(180, 500) train_X 

Time to transform: 1.9670898914337158 seconds
Time to fit classifier on train: 43.55842113494873 seconds
Time to predict: 3.1606154441833496 seconds
0.5722222222222222 accuracy for <features.randomized_sig.RandomizedSignature object at 0x7f301ee2dcd0>


In [7]:
train_test_EC(
    RandomizedSignature(
        n_features=500,
        activation="tanh"
    ),
    device="cuda",
)

(180, 500) train_X 

Time to transform: 2.150810956954956 seconds
Time to fit classifier on train: 43.39476466178894 seconds
Time to predict: 2.971315622329712 seconds
0.5555555555555556 accuracy for <features.randomized_sig.RandomizedSignature object at 0x7f301e3b72d0>


In [ ]:
train_test_EC(
    SigVanillaTensorizedRandProj(
        n_features=500,
        trunc_level=2,
    ),
    device="cpu",
)
# Time to transform: 0.37505316734313965 seconds
# Time to fit classifier on train: 608.9609467983246 seconds
# Time to predict: 39.79999351501465 seconds
# 0.3612167300380228 accuracy for SigVanillaTensorizedRandProj(n_features=5000)


In [ ]:
train_test_EC(
    SigRBFTensorizedRandProj(
        n_features=500,
        trunc_level=3,
        sigma=10.0,
    ),
    device="cuda",
)

In [ ]:
train_test_EC(
    SigTransform(
        trunc_level=2,
    ),
    device="cuda",
)

In [ ]:
train_test_EC(
    LogSigTransform(
        trunc_level=2,
    ),
    device="cpu",
)